# Using Interpretability Dashboard with Employee Attrition Data

This notebook illustrates creating explanations for a binary classification model, employee attrition classification, that uses one to one and one to many feature transformations from raw data to engineered features. It will showcase raw feature transformations with three tabular data explainers: TabularExplainer (SHAP), MimicExplainer (global surrogate), and PFIExplainer.

Problem: Employee attrition classification with scikit-learn (run model explainer locally)

1. Transform raw features to engineered features.
2. Train a classification model using Scikit-learn.
3. Run 'explain_model' globally and locally with full dataset.
4. Visualize the global and local explanations with the interpretability visualization dashboard.

## Install Required Packages

In [1]:
# %pip install --upgrade interpret-community

  Using cached interpret_community-0.17.1-py3-none-any.whl (5.9 MB)
  Using cached shap-0.34.0-cp37-cp37m-win_amd64.whl (298 kB)
  Using cached interpret_core-0.2.4-py3-none-any.whl (5.8 MB)
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


After installing packages, you must close and reopen the notebook as well as restarting the kernel.

## Explain

### Run model explainer at training time

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
import zipfile

### Load the employee attrition data

In [3]:
outdirname = 'dataset.6.21.19'
zipfilename = outdirname + '.zip'
urlretrieve('https://publictestdatasets.blob.core.windows.net/data/' + zipfilename, zipfilename)
with zipfile.ZipFile(zipfilename, 'r') as unzip:
    unzip.extractall('.')
attritionData = pd.read_csv('./WA_Fn-UseC_-HR-Employee-Attrition.csv')

# Dropping Employee count as all values are 1 and hence attrition is independent of this feature
attritionData = attritionData.drop(['EmployeeCount'], axis=1)
# Dropping Employee Number since it is merely an identifier
attritionData = attritionData.drop(['EmployeeNumber'], axis=1)
attritionData = attritionData.drop(['Over18'], axis=1)

# Since all values are 80
attritionData = attritionData.drop(['StandardHours'], axis=1)

# Converting target variables from string to numerical values
target_map = {'Yes': 0, 'No': 1}
attritionData["Attrition_numerical"] = attritionData["Attrition"].apply(lambda x: target_map[x])
target = attritionData["Attrition_numerical"]

attritionXData = attritionData.drop(['Attrition_numerical', 'Attrition'], axis=1)

In [4]:
# Split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(attritionXData, 
                                                    target, 
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=target)

In [5]:
# Creating dummy columns for each categorical feature
categorical = []
for col, value in attritionXData.iteritems():
    if value.dtype == 'object':
        categorical.append(col)
        
# Store the numerical columns in a list numerical
numerical = attritionXData.columns.difference(categorical)        

### Transform raw features

You can explain raw features by either using a `sklearn.compose.ColumnTransformer` or a list of fitted transformer tuples. The cell below uses `sklearn.compose.ColumnTransformer`. In case you want to run the example with the list of fitted transformer tuples, comment the cell below and uncomment the cell that follows after. 

In [6]:
from sklearn.compose import ColumnTransformer

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', LGBMClassifier())])


### Train a LightGBM classification model, which you want to explain

In [7]:
model = clf.fit(x_train, y_train)

### Explain your model predictions

In [8]:
# clf.steps[-1][1] returns the trained classification model

# 1. Using SHAP TabularExplainer
from interpret.ext.blackbox import TabularExplainer
explainer = TabularExplainer(clf.steps[-1][1], 
                             initialization_examples=x_train, 
                             features=attritionXData.columns, 
                             classes=['Leaving', 'Staying'], 
                             transformations=transformations)


# 2. Using MimicExplainer
# augment_data is optional and if true, oversamples the initialization examples to improve surrogate model accuracy to fit original model.  Useful for high-dimensional data where the number of rows is less than the number of columns. 
# max_num_of_augmentations is optional and defines max number of times we can increase the input data size.
# LGBMExplainableModel can be replaced with LinearExplainableModel, SGDExplainableModel, or DecisionTreeExplainableModel
# from interpret.ext.blackbox import MimicExplainer
# from interpret.ext.glassbox import LGBMExplainableModel
# explainer = MimicExplainer(clf.steps[-1][1], 
#                            x_train, 
#                            LGBMExplainableModel, 
#                            augment_data=True, 
#                            max_num_of_augmentations=10, 
#                            features=attritionXData.columns, 
#                            classes=["Leaving", "Staying"], 
#                            transformations=transformations)





# 3. Using PFIExplainer
# Use the parameter "metric" to pass a metric name or function to evaluate the permutation. 
# Note that if a metric function is provided a higher value must be better.
# Otherwise, take the negative of the function or set the parameter "is_error_metric" to True.
# Default metrics: 
# F1 Score for binary classification, F1 Score with micro average for multiclass classification and
# Mean absolute error for regression
# from interpret.ext.blackbox import PFIExplainer
# explainer = PFIExplainer(clf.steps[-1][1], 
#                          features=x_train.columns, 
#                          transformations=transformations,
#                          classes=["Leaving", "Staying"])


Could not import LIME, required for LIMEExplainer
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


### Generate global explanations
Explain overall model predictions (global explanation)

In [9]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# x_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
global_explanation = explainer.explain_global(x_test)

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [10]:
# Print out a dictionary that holds the sorted feature importance names and values
print('global importance rank: {}'.format(global_explanation.get_feature_importance_dict()))

global importance rank: {'OverTime': 0.8671180034119259, 'StockOptionLevel': 0.5935162276276619, 'MonthlyIncome': 0.5077163466162175, 'EnvironmentSatisfaction': 0.38370025137840846, 'NumCompaniesWorked': 0.3621168979475014, 'DailyRate': 0.3294469364706887, 'DistanceFromHome': 0.32849406911313805, 'YearsWithCurrManager': 0.32023648645055924, 'JobRole': 0.2909562918384063, 'BusinessTravel': 0.2817429228713378, 'JobSatisfaction': 0.27726380403494433, 'JobInvolvement': 0.27331413643868785, 'Age': 0.2646025452416837, 'EducationField': 0.219575214376005, 'TotalWorkingYears': 0.18424426064629487, 'YearsSinceLastPromotion': 0.1717552999308561, 'PercentSalaryHike': 0.16082737771731798, 'RelationshipSatisfaction': 0.1588264184437708, 'MonthlyRate': 0.15783505069727408, 'WorkLifeBalance': 0.15720113204387484, 'HourlyRate': 0.14775935354581018, 'MaritalStatus': 0.14756350726759088, 'YearsAtCompany': 0.14555452717451922, 'JobLevel': 0.14419841695938582, 'TrainingTimesLastYear': 0.13421152646827703,

### Generate local explanations
Explain local data points (individual instances)

In [11]:
# You can pass a specific data point or a group of data points to the explain_local function
# E.g., Explain the first data point in the test set
instance_num = 1
local_explanation = explainer.explain_local(x_test[:instance_num])

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [12]:
# Get the prediction for the first member of the test set and explain why model made that prediction
prediction_value = clf.predict(x_test)[instance_num]

sorted_local_importance_values = local_explanation.get_ranked_local_values()[prediction_value]
sorted_local_importance_names = local_explanation.get_ranked_local_names()[prediction_value]

In [13]:
print('local importance values: {}'.format(sorted_local_importance_values))
print('local importance names: {}'.format(sorted_local_importance_names))

local importance values: [[0.9872782509670005, 0.6098483362087465, 0.49360776366437137, 0.3958696927838185, 0.2511517496176293, 0.22759400856616324, 0.14984831748112823, 0.11269249192142855, 0.07753491402246747, 0.05533656450429863, 0.04854240809166762, 0.04119399537275928, 0.03863692290194614, 0.0116183804145255, 0.0, -0.000829737976490213, -0.04020280699078922, -0.04137092837165063, -0.0870175410207079, -0.09508504778043447, -0.10265623437460784, -0.11253826053335347, -0.1570362620333481, -0.19090440616256293, -0.27674855535331966, -0.32846991637164863, -0.6214704096223682, -0.6348852178684176, -1.1339969326551331, -1.1737571991243123]]
local importance names: [['OverTime', 'MaritalStatus', 'StockOptionLevel', 'EnvironmentSatisfaction', 'DistanceFromHome', 'BusinessTravel', 'TotalWorkingYears', 'WorkLifeBalance', 'JobInvolvement', 'YearsSinceLastPromotion', 'Education', 'TrainingTimesLastYear', 'RelationshipSatisfaction', 'PercentSalaryHike', 'PerformanceRating', 'YearsInCurrentRole'

## Visualize
Load the interpretability visualization dashboard

In [14]:
from raiwidgets import ExplanationDashboard

ModuleNotFoundError: No module named 'raiwidgets'

In [ ]:
ExplanationDashboard(global_explanation, model, dataset=x_test, true_y=y_test.values)